In [ ]:
# Import python packages
import re
import json
import streamlit as st
import altair as alt
import pandas as pd
from snowflake.snowpark.functions import col, lit


# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
def string_query_parser(sql_text: str) -> list[str]:
    """Parse a string which contains queries separated by ;
    and return a list of query strings."""
    queries: list[str] = []

    if sql_text:
        # Split the cleaned SQL text into individual statements
        statements = re.split(r';\s*', sql_text)

        queries = [stmt for stmt in statements if stmt]

    return queries

In [ ]:
def run_sql_queries(db_schema: str, queries: list[str], wh_size: str, disable_cache_query: bool) -> list[str]:
    # disable cache query for the session if needed
    if disable_cache_query:
        session.sql(f"ALTER SESSION SET USE_CACHED_RESULT = FALSE").collect()

    # set the database schema
    session.use_schema(db_schema)

    # change the warehouse size"
    session.sql(f"ALTER WAREHOUSE ANALYSIS SET WAREHOUSE_SIZE = '{wh_size}'").collect()
    query_ids: list[str] = []

    # execute queries in parallel
    for query in queries:
        # exec the query in async mode
        job = session.sql(query).collect_nowait()
        # query the query id
        query_ids.append(job.query_id)
    return query_ids

In [ ]:
def wait_till_end_and_scale_down(query_ids: str) -> None:
    # wait till copy async jobs ended on session
    while True:
        # check if there are copy into queries in running status for the current session
        df = (
            session.table_function(
                "MEETUP_GDDP.INFORMATION_SCHEMA.QUERY_HISTORY_BY_SESSION",
                result_limit=lit(10000)
            )
            .filter(
                ~(col("EXECUTION_STATUS").in_(["SUCCESS", "FAILED_WITH_ERROR", "FAILED_WITH_INCIDENT",
                                              "ABORTED", "DISCONNECTED"]))
                & (col("QUERY_ID").in_(query_ids))
            )
        )

        if df.count() == 0:
            break

    # scale down the warehouse
    session.sql(f"ALTER WAREHOUSE ANALYSIS SET WAREHOUSE_SIZE = 'X-SMALL'").collect()

    # reset use cache result as true
    session.sql(f"ALTER SESSION SET USE_CACHED_RESULT = TRUE").collect()

In [ ]:
def display_results(query_ids: list[str]) -> None:
    from snowflake.snowpark.functions import col, regexp_extract
    # get queries stats by queries
    df = (
            session.table_function(
                "MEETUP_GDDP.INFORMATION_SCHEMA.QUERY_HISTORY_BY_SESSION",
                result_limit=lit(10000)
            )
            .filter(
                (col("QUERY_ID").in_(query_ids))
            )
            .select(
                [
                    "SESSION_ID",
                    "QUERY_ID",
                    "QUERY_TEXT",
                    "WAREHOUSE_SIZE",
                    "START_TIME",
                    "END_TIME",
                    (col("TOTAL_ELAPSED_TIME") / 1000).alias("TOTAL_ELAPSED_TIME_SECONDS"),
                    (col("COMPILATION_TIME") / 1000).alias("COMPILATION_TIME_SECONDS"),
                    (col("QUEUED_OVERLOAD_TIME") / 1000).alias("QUEUED_OVERLOAD_TIME_SECONDS"),
                    "ROWS_PRODUCED"
                ]
            )
        )

    # Collect the DataFrame to Pandas
    df_pandas = df.to_pandas()

    # Display the total elapsed time as a Streamlit metric
    total_duration_seconds = (pd.to_datetime(df_pandas["END_TIME"]).max()
                              - pd.to_datetime(df_pandas["START_TIME"]).min()).total_seconds()
    st.metric(label="Total Elapsed Time (seconds)",
              value=f"{total_duration_seconds:.2f}")

    # Create the Altair bar chart
    bar_chart = (
        alt.Chart(df_pandas)
        .mark_bar(color="#872D60")
        .encode(
            x=alt.X("QUERY_ID:N", title=None),
            y=alt.Y("TOTAL_ELAPSED_TIME_SECONDS:Q", title="Duration in seconds)"),
            xOffset=alt.XOffset("WAREHOUSE_SIZE:N")
        )
    )

    # Display the Altair chart in Streamlit
    st.altair_chart(bar_chart, use_container_width=True)

    # Display the DataFrame in Streamlit
    st.write(df_pandas)

In [ ]:
def get_tpch_queries() -> str:
    return """
    -- TPC-H 1
    select
    	l_returnflag,
    	l_linestatus,
    	sum(l_quantity) as sum_qty,
    	sum(l_extendedprice) as sum_base_price,
    	sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
    	sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
    	avg(l_quantity) as avg_qty,
    	avg(l_extendedprice) as avg_price,
    	avg(l_discount) as avg_disc,
    	count(*) as count_order
    from
    	lineitem
    where
    	l_shipdate <= DATEADD(day, 90, '1998-12-01')
    group by
    	l_returnflag,
    	l_linestatus
    order by
    	l_returnflag,
    	l_linestatus;

    -- TPC-H 2
    select
    	s_acctbal,
    	s_name,
    	n_name,
    	p_partkey,
    	p_mfgr,
    	s_address,
    	s_phone,
    	s_comment
    from
    	part,
    	supplier,
    	partsupp,
    	nation,
    	region
    where
    	p_partkey = ps_partkey
    	and s_suppkey = ps_suppkey
    	and p_size = 15
    	and p_type like '%BRASS'
    	and s_nationkey = n_nationkey
    	and n_regionkey = r_regionkey
    	and r_name = 'EUROPE'
    	and ps_supplycost = (
    		select
    			min(ps_supplycost)
    		from
    			partsupp,
    			supplier,
    			nation,
    			region
    		where
    			p_partkey = ps_partkey
    			and s_suppkey = ps_suppkey
    			and s_nationkey = n_nationkey
    			and n_regionkey = r_regionkey
    			and r_name = 'EUROPE'
    	)
    order by
    	s_acctbal desc,
    	n_name,
    	s_name,
    	p_partkey LIMIT 100;

    -- TPC-H 3
    select
    	l_orderkey,
    	sum(l_extendedprice * (1 - l_discount)) as revenue,
    	o_orderdate,
    	o_shippriority
    from
    	customer,
    	orders,
    	lineitem
    where
    	c_mktsegment = 'BUILDING'
    	and c_custkey = o_custkey
    	and l_orderkey = o_orderkey
    	and o_orderdate < date '1995-03-15'
    	and l_shipdate > date '1995-03-15'
    group by
    	l_orderkey,
    	o_orderdate,
    	o_shippriority
    order by
    	revenue desc,
    	o_orderdate limit 10;

    -- TPC-H 4
    select
    	o_orderpriority,
    	count(*) as order_count
    from
    	orders
    where
    	o_orderdate >= date '1993-07-01'
    	AND o_orderdate < DATEADD(month, 3, '1993-07-01')
    	and exists (
    		select
    			*
    		from
    			lineitem
    		where
    			l_orderkey = o_orderkey
    			and l_commitdate < l_receiptdate
    	)
    group by
    	o_orderpriority
    order by
    	o_orderpriority;

    -- TPC-H 5
    select
    	n_name,
    	sum(l_extendedprice * (1 - l_discount)) as revenue
    from
    	customer,
    	orders,
    	lineitem,
    	supplier,
    	nation,
    	region
    where
    	c_custkey = o_custkey
    	and l_orderkey = o_orderkey
    	and l_suppkey = s_suppkey
    	and c_nationkey = s_nationkey
    	and s_nationkey = n_nationkey
    	and n_regionkey = r_regionkey
    	and r_name = 'ASIA'
    	and o_orderdate >= date '1994-01-01'
    	AND o_orderdate < DATEADD(year, 1, '1994-01-01')
    group by
    	n_name
    order by
    	revenue desc;

    -- TPC-H 6
    select
    	sum(l_extendedprice * l_discount) as revenue
    from
    	snowflake_sample_data.tpch_sf1.lineitem
    where
    	l_shipdate >= date '1994-01-01'
    	AND l_shipdate < DATEADD(year, 1, '1994-01-01')
    	and l_discount between .06 - 0.01 and .06 + 0.01
    	and l_quantity < 24;

    -- TPC-H 7
    select
    	supp_nation,
    	cust_nation,
    	l_year,
    	sum(volume) as revenue
    from
    	(
    		select
    			n1.n_name as supp_nation,
    			n2.n_name as cust_nation,
    			extract(year from l_shipdate) as l_year,
    			l_extendedprice * (1 - l_discount) as volume
    		from
    			supplier,
    			lineitem,
    			orders,
    			customer,
    			nation n1,
    			nation n2
    		where
    			s_suppkey = l_suppkey
    			and o_orderkey = l_orderkey
    			and c_custkey = o_custkey
    			and s_nationkey = n1.n_nationkey
    			and c_nationkey = n2.n_nationkey
    			and (
    				(n1.n_name = 'FRANCE' and n2.n_name = 'GERMANY')
    				or (n1.n_name = 'GERMANY' and n2.n_name = 'FRANCE')
    			)
    			and l_shipdate between date '1995-01-01' and date '1996-12-31'
    	) as shipping
    group by
    	supp_nation,
    	cust_nation,
    	l_year
    order by
    	supp_nation,
    	cust_nation,
    	l_year;

    -- TPC-H 8
    select
    	o_year,
    	sum(case
    		when nation = 'BRAZIL' then volume
    		else 0
    	end) / sum(volume) as mkt_share
    from
    	(
    		select
    			extract(year from o_orderdate) as o_year,
    			l_extendedprice * (1 - l_discount) as volume,
    			n2.n_name as nation
    		from
    			part,
    			supplier,
    			lineitem,
    			orders,
    			customer,
    			nation n1,
    			nation n2,
    			region
    		where
    			p_partkey = l_partkey
    			and s_suppkey = l_suppkey
    			and l_orderkey = o_orderkey
    			and o_custkey = c_custkey
    			and c_nationkey = n1.n_nationkey
    			and n1.n_regionkey = r_regionkey
    			and r_name = 'AMERICA'
    			and s_nationkey = n2.n_nationkey
    			and o_orderdate between date '1995-01-01' and date '1996-12-31'
    			and p_type = 'ECONOMY ANODIZED STEEL'
    	) as all_nations
    group by
    	o_year
    order by
    	o_year;

    -- TPC-H 9
    select
    	nation,
    	o_year,
    	sum(amount) as sum_profit
    from
    	(
    		select
    			n_name as nation,
    			extract(year from o_orderdate) as o_year,
    			l_extendedprice * (1 - l_discount) - ps_supplycost * l_quantity as amount
    		from
    			part,
    			supplier,
    			lineitem,
    			partsupp,
    			orders,
    			nation
    		where
    			s_suppkey = l_suppkey
    			and ps_suppkey = l_suppkey
    			and ps_partkey = l_partkey
    			and p_partkey = l_partkey
    			and o_orderkey = l_orderkey
    			and s_nationkey = n_nationkey
    			and p_name like '%green%'
    	) as profit
    group by
    	nation,
    	o_year
    order by
    	nation,
    	o_year desc;

    -- TPC-H 10
    select
    	c_custkey,
    	c_name,
    	sum(l_extendedprice * (1 - l_discount)) as revenue,
    	c_acctbal,
    	n_name,
    	c_address,
    	c_phone,
    	c_comment
    from
    	customer,
    	orders,
    	lineitem,
    	nation
    where
    	c_custkey = o_custkey
    	and l_orderkey = o_orderkey
    	and o_orderdate >= date '1993-10-01'
    	AND o_orderdate < DATEADD(month, 3, '1993-10-01')
    	and l_returnflag = 'R'
    	and c_nationkey = n_nationkey
    group by
    	c_custkey,
    	c_name,
    	c_acctbal,
    	c_phone,
    	n_name,
    	c_address,
    	c_comment
    order by
    	revenue desc limit 20;

    -- TPC-H 11
    select
    	ps_partkey,
    	sum(ps_supplycost * ps_availqty) as value
    from
    	partsupp,
    	supplier,
    	nation
    where
    	ps_suppkey = s_suppkey
    	and s_nationkey = n_nationkey
    	and n_name = 'GERMANY'
    group by
    	ps_partkey having
    		sum(ps_supplycost * ps_availqty) > (
    			select
    				sum(ps_supplycost * ps_availqty) * 0.0001000000
    			from
    				partsupp,
    				supplier,
    				nation
    			where
    				ps_suppkey = s_suppkey
    				and s_nationkey = n_nationkey
    				and n_name = 'GERMANY'
    		)
    order by
    	value desc;

    -- TPC-H 12
    select
    	l_shipmode,
    	sum(case
    		when o_orderpriority = '1-URGENT'
    			or o_orderpriority = '2-HIGH'
    			then 1
    		else 0
    	end) as high_line_count,
    	sum(case
    		when o_orderpriority <> '1-URGENT'
    			and o_orderpriority <> '2-HIGH'
    			then 1
    		else 0
    	end) as low_line_count
    from
    	orders,
    	lineitem
    where
    	o_orderkey = l_orderkey
    	and l_shipmode in ('MAIL', 'SHIP')
    	and l_commitdate < l_receiptdate
    	and l_shipdate < l_commitdate
    	and l_receiptdate >= date '1994-01-01'
    	AND o_orderdate < DATEADD(year, 1, '1994-01-01')
    group by
    	l_shipmode
    order by
    	l_shipmode;

    -- TPC-H 13
    select
    	c_count,
    	count(*) as custdist
    from
    	(
    		select
    			c_custkey,
    			count(o_orderkey)
    		from
    			customer left outer join orders on
    				c_custkey = o_custkey
    				and o_comment not like '%special%requests%'
    		group by
    			c_custkey
    	) as c_orders (c_custkey, c_count)
    group by
    	c_count
    order by
    	custdist desc,
    	c_count desc;

    -- TPC-H 14
    select
    	100.00 * sum(case
    		when p_type like 'PROMO%'
    			then l_extendedprice * (1 - l_discount)
    		else 0
    	end) / sum(l_extendedprice * (1 - l_discount)) as promo_revenue
    from
    	lineitem,
    	part
    where
    	l_partkey = p_partkey
    	and l_shipdate >= date '1995-09-01'
    	AND l_shipdate < DATEADD(month, 1, '1995-09-01');

    -- TPC-H 15
    with revenue_view as (
      select
        l_suppkey as supplier_no,
        sum(l_extendedprice * (1 - l_discount)) as total_revenue
      from
        lineitem
      where
        l_shipdate >= '1996-01-01'
        and l_shipdate < '1996-04-01'
      group by
        l_suppkey)
    select
      s_suppkey,
      s_name,
      s_address,
      s_phone,
      total_revenue
    from
      supplier,
      revenue_view
    where
      s_suppkey = supplier_no
      and total_revenue = (
        select
          max(total_revenue)
        from
          revenue_view
        )
    order by
      s_suppkey;

    -- TPC-H 16
    select
    	p_brand,
    	p_type,
    	p_size,
    	count(distinct ps_suppkey) as supplier_cnt
    from
    	partsupp,
    	part
    where
    	p_partkey = ps_partkey
    	and p_brand <> 'Brand#45'
    	and p_type not like 'MEDIUM POLISHED%'
    	and p_size in (49, 14, 23, 45, 19, 3, 36, 9)
    	and ps_suppkey not in (
    		select
    			s_suppkey
    		from
    			supplier
    		where
    			s_comment like '%Customer%Complaints%'
    	)
    group by
    	p_brand,
    	p_type,
    	p_size
    order by
    	supplier_cnt desc,
    	p_brand,
    	p_type,
    	p_size;

    -- TPC-H 17
    select
    	sum(l_extendedprice) / 7.0 as avg_yearly
    from
    	lineitem,
    	part
    where
    	p_partkey = l_partkey
    	and p_brand = 'Brand#23'
    	and p_container = 'MED BOX'
    	and l_quantity < (
    		select
    			0.2 * avg(l_quantity)
    		from
    			lineitem
    		where
    			l_partkey = p_partkey
    	);

    -- TPC-H 18
    select
    	c_name,
    	c_custkey,
    	o_orderkey,
    	o_orderdate,
    	o_totalprice,
    	sum(l_quantity)
    from
    	customer,
    	orders,
    	lineitem
    where
    	o_orderkey in (
    		select
    			l_orderkey
    		from
    			lineitem
    		group by
    			l_orderkey having
    				sum(l_quantity) > 300
    	)
    	and c_custkey = o_custkey
    	and o_orderkey = l_orderkey
    group by
    	c_name,
    	c_custkey,
    	o_orderkey,
    	o_orderdate,
    	o_totalprice
    order by
    	o_totalprice desc,
    	o_orderdate
    limit 100;

    -- TPC-H 19
    select
    	sum(l_extendedprice* (1 - l_discount)) as revenue
    from
    	lineitem,
    	part
    where
    	(
    		p_partkey = l_partkey
    		and p_brand = 'Brand#12'
    		and p_container in ('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG')
    		and l_quantity >= 1 and l_quantity <= 1 + 10
    		and p_size between 1 and 5
    		and l_shipmode in ('AIR', 'AIR REG')
    		and l_shipinstruct = 'DELIVER IN PERSON'
    	)
    	or
    	(
    		p_partkey = l_partkey
    		and p_brand = 'Brand#23'
    		and p_container in ('MED BAG', 'MED BOX', 'MED PKG', 'MED PACK')
    		and l_quantity >= 10 and l_quantity <= 10 + 10
    		and p_size between 1 and 10
    		and l_shipmode in ('AIR', 'AIR REG')
    		and l_shipinstruct = 'DELIVER IN PERSON'
    	)
    	or
    	(
    		p_partkey = l_partkey
    		and p_brand = 'Brand#34'
    		and p_container in ('LG CASE', 'LG BOX', 'LG PACK', 'LG PKG')
    		and l_quantity >= 20 and l_quantity <= 20 + 10
    		and p_size between 1 and 15
    		and l_shipmode in ('AIR', 'AIR REG')
    		and l_shipinstruct = 'DELIVER IN PERSON'
    	);

    -- TPC-H 20
    select
    	s_name,
    	s_address
    from
    	supplier,
    	nation
    where
    	s_suppkey in (
    		select
    			ps_suppkey
    		from
    			partsupp
    		where
    			ps_partkey in (
    				select
    					p_partkey
    				from
    					part
    				where
    					p_name like 'forest%'
    			)
    			and ps_availqty > (
    				select
    					0.5 * sum(l_quantity)
    				from
    					lineitem
    				where
    					l_partkey = ps_partkey
    					and l_suppkey = ps_suppkey
    					and l_shipdate >= date '1994-01-01'
                    	AND l_shipdate < DATEADD(year, 1, '1994-01-01')
    			)
    	)
    	and s_nationkey = n_nationkey
    	and n_name = 'CANADA'
    order by
    	s_name;

    -- TPC-H 21
    select
    	s_name,
    	count(*) as numwait
    from
    	supplier,
    	lineitem l1,
    	orders,
    	nation
    where
    	s_suppkey = l1.l_suppkey
    	and o_orderkey = l1.l_orderkey
    	and o_orderstatus = 'F'
    	and l1.l_receiptdate > l1.l_commitdate
    	and exists (
    		select
    			*
    		from
    			lineitem l2
    		where
    			l2.l_orderkey = l1.l_orderkey
    			and l2.l_suppkey <> l1.l_suppkey
    	)
    	and not exists (
    		select
    			*
    		from
    			lineitem l3
    		where
    			l3.l_orderkey = l1.l_orderkey
    			and l3.l_suppkey <> l1.l_suppkey
    			and l3.l_receiptdate > l3.l_commitdate
    	)
    	and s_nationkey = n_nationkey
    	and n_name = 'SAUDI ARABIA'
    group by
    	s_name
    order by
    	numwait desc,
    	s_name limit 100;

    -- TPC-H 22
    select
    	cntrycode,
    	count(*) as numcust,
    	sum(c_acctbal) as totacctbal
    from
    	(
    		select
          SUBSTRING(c_phone, 1, 2) AS cntrycode,
    			c_acctbal
    		from
    			customer
    		where
              SUBSTRING(c_phone, 1, 2) IN
    				('13', '31', '23', '29', '30', '18', '17')
    			and c_acctbal > (
    				select
    					avg(c_acctbal)
    				from
    					customer
    				where
    					c_acctbal > 0.00
                        AND SUBSTRING(c_phone, 1, 2) IN
    						('13', '31', '23', '29', '30', '18', '17')
    			)
    			and not exists (
    				select
    					*
    				from
    					orders
    				where
    					o_custkey = c_custkey
    			)
    	) as custsale
    group by
    	cntrycode
    order by
    	cntrycode
    """

In [ ]:
# file format and base location on external stage
# wh_size = "X-Large"
wh_size = "X-Small"
db_schema = "MEETUP_GDDP.TPCH_SF100"
disable_cache_query = False
view_results = True

queries = string_query_parser(get_tpch_queries())
query_ids = run_sql_queries(db_schema, queries, wh_size, disable_cache_query)
wait_till_end_and_scale_down(query_ids)
if view_results:
    display_results(query_ids)